In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import glob
import os
import ast # converts string of list -> list
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import spacy 
from scipy.spatial.distance import cosine

nlp = spacy.load('en', disable=['parser', 'ner'])

from spacy.tokens import Doc

def custom_tokenizer(tokens):
    # with this:
    return Doc(nlp.vocab, tokens)

nlp.tokenizer = custom_tokenizer


In [2]:
################# MODEL PARAMETERS #####################
path_we = "/home/adrian/PhD/Data/FastText_embedding_20190703/ft_wordembeddings_dim300_minCount5_URL-User-toConstant_iter10_20190703"
path_we = "/Users/adrianahne/PhD/Data/FastText_embeddings/ft_wordembeddings_dim300_minCount5_URL-User-toConstant_iter10_20190703"
dataPath = "result_cause_effect_prediction"
csv_files = glob.glob(os.path.join(dataPath, "*.csv"))
len(csv_files)
stopword_list = [word for word in stopwords.words('english') if word not in ["no", "nor", "not", "don", "don't", 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]]



In [3]:

################## LOAD DATA ######################
tuples = []
for file in csv_files:
    with open(file, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if i == 0: # header
                if line.endswith("\n"):
                    line = line[:-2]
                header = line.split(",")[1:]
            else:
                index, ll = line.split(",", 1)
                ll, io_tags = ll.rsplit("[", 1)
                if io_tags.endswith('\n'):
                    io_tags = io_tags[:-2] # remove \n
                elif io_tags.endswith('"'):
                    io_tags = io_tags[:-1]
                io_tags = ast.literal_eval("["+io_tags)

                text, tokenized = ll.rsplit(',"[', 1)
                tokenized = tokenized[:-3].replace('""', '"')
                tokenized = ast.literal_eval("["+tokenized)

                if text.endswith(',"'):
                    text = text[:-2]
                if text.startswith('"'):
                    text = text[1:]
                if text.endswith('"'):
                    text = text[:-1]

                tuples.append((index, text, tokenized, io_tags))
            
print(len(tuples))

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(tuples, columns=["index","text", "tokenized", "io_tags"])
df.head()

In [27]:
def extract_causes_effects(row):
    """ Extracts causes and effects from the io_tags column and saves them in new columns """
    #print()
    #print(row["text"])
    #for tag, tok in zip(row["io_tags"], row["tokenized"]):
    #    print(tok, tag)
    if len(row["tokenized"]) != len(row["io_tags"]):
        print("ERROR: Tokenized and tags are not of same length!")
    
    causes = ""
    effects = ""
    last_tag_cause_index = -2
    last_tag_effect_index = -2
    for i, tag in enumerate(row.io_tags):
        if tag == "I-C":
            if i == last_tag_cause_index + 1: # True, if cause consist of several consecutive words
                causes += ","+row["tokenized"][i]
            else: # cause is a new separate cause
                causes += ";"+row["tokenized"][i]
            last_tag_cause_index = i # helps to test if causes and effects consist of CONSECUTIVE words

        elif tag == "I-E":
            if i == last_tag_effect_index + 1: # True, if cause consist of several consecutive words
                effects += ","+row["tokenized"][i]
            else: # cause is a new separate cause
                effects += ";"+row["tokenized"][i]
            last_tag_effect_index = i # helps to test if causes and effects consist of CONSECUTIVE words
    
    if causes.startswith(",") or causes.startswith(";"):
        causes = causes[1:]
    if effects.startswith(",") or effects.startswith(";"):
        effects = effects[1:]        
    #print("causes:", causes)
    #print("effects:", effects)
    
    

    
    return pd.Series([row.index, row.text, row.tokenized, row.io_tags, causes, effects]
                     , index=["index", "text", "tokenized", "io_tags", "causes", "effects"])

df_with_causes_effects = df.apply(extract_causes_effects, axis=1)

In [29]:
df_with_causes_effects.head()

,index,text,tokenized,io_tags,causes,effects
0,"Index(['index', 'text', 'tokenized', 'io_tags'...",@USER There will be no shortages of insulin or...,"[@USER, There, will, be, no, shortages, of, in...","[O, O, O, O, O, I-C, I-C, I-C, O, O, O, O, O, ...","shortages,of,insulin",worried
1,"Index(['index', 'text', 'tokenized', 'io_tags'...",Back to the #robotic life and I 'm good with t...,"[Back, to, the, #robotic, life, and, I, 'm, go...","[O, O, O, O, O, O, O, O, O, O, O, I-C, I-C, I-...","#libre,#freestylelibre,#abbott,#t1d,#diabetes,...",
2,"Index(['index', 'text', 'tokenized', 'io_tags'...",@USER I can't take such medications anymore ca...,"[@USER, I, can't, take, such, medications, any...","[O, O, O, O, O, O, O, O, O, O, O, I-C, I-C, O,...","diabetic,condition",
3,"Index(['index', 'text', 'tokenized', 'io_tags'...",If Brexit kills me because insulin is made in ...,"[If, Brexit, kills, me, because, insulin, is, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",,
4,"Index(['index', 'text', 'tokenized', 'io_tags'...","cool thing here , im only not diabetic anymore...","[cool, thing, here, ,, im, only, not, diabetic...","[O, O, O, O, O, O, I-C, I-C, O, O, O, O, O, O,...","not,diabetic",


In [31]:
###################### STORE RESULTS FOR EASIER ACCESS #####################
#del df_with_causes_effects["index"]
#df_with_causes_effects.to_parquet("result_cause_effect_prediction_all.parquet")

In [3]:
########### LOAD DATA ###################
df_with_causes_effects = pd.read_parquet("result_cause_effect_prediction_all.parquet")
df_with_causes_effects.shape

(265328, 5)

In [4]:
#################### Only consider tweets with both cause and effect ###############################
df_new = df_with_causes_effects[(df_with_causes_effects.causes.str.len() > 0 ) & (df_with_causes_effects.effects.str.len() > 0)]
df_new.reset_index(drop=True, inplace=True)
print(df_new.shape)
df_new.head()

(96676, 5)


,text,tokenized,io_tags,causes,effects
0,@USER There will be no shortages of insulin or...,"[@USER, There, will, be, no, shortages, of, in...","[O, O, O, O, O, I-C, I-C, I-C, O, O, O, O, O, ...","shortages,of,insulin",worried
1,someone 's a type II diabetic and they basical...,"[someone, 's, a, type, II, diabetic, and, they...","[O, O, O, I-C, I-C, I-C, O, O, O, I-E, I-E, I-...","type,II,diabetic","can't,eat,food"
2,"I would use it to fix my truck , keep bills pa...","[I, would, use, it, to, fix, my, truck, ,, kee...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",diabetes,"deformed,my,feet"
3,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,"get,rid,of,the,insulin;diet"
4,Just drunk that nasty ass shit for this glucos...,"[Just, drunk, that, nasty, ass, shit, for, thi...","[O, O, O, O, O, O, O, O, I-C, I-C, O, O, O, I-...","glucose,test","feel,sick,:nauseated_face:"


In [5]:
################## SPLIT CAUSES AND EFFECTS s.t. each row has one cause and one effect #######################
df_new = df_new.assign(causes=df_new["causes"].str.split(";")).explode("causes")
df_new = df_new.assign(effects=df_new["effects"].str.split(";")).explode("effects")
df_new.reset_index(drop=True, inplace=True)
print(df_new.shape)
df_new.head(20)

(149798, 5)


,text,tokenized,io_tags,causes,effects
0,@USER There will be no shortages of insulin or...,"[@USER, There, will, be, no, shortages, of, in...","[O, O, O, O, O, I-C, I-C, I-C, O, O, O, O, O, ...","shortages,of,insulin",worried
1,someone 's a type II diabetic and they basical...,"[someone, 's, a, type, II, diabetic, and, they...","[O, O, O, I-C, I-C, I-C, O, O, O, I-E, I-E, I-...","type,II,diabetic","can't,eat,food"
2,"I would use it to fix my truck , keep bills pa...","[I, would, use, it, to, fix, my, truck, ,, kee...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",diabetes,"deformed,my,feet"
3,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,"get,rid,of,the,insulin"
4,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,diet
5,Just drunk that nasty ass shit for this glucos...,"[Just, drunk, that, nasty, ass, shit, for, thi...","[O, O, O, O, O, O, O, O, I-C, I-C, O, O, O, I-...","glucose,test","feel,sick,:nauseated_face:"
6,Me and my sister literally CRIED for him when ...,"[Me, and, my, sister, literally, CRIED, for, h...","[O, O, O, O, O, I-E, O, O, O, O, O, O, O, O, I...",diabetes,CRIED
7,@USER @USER I know people with severe depressi...,"[@USER, @USER, I, know, people, with, severe, ...","[O, O, O, O, O, O, O, I-E, O, O, O, I-C, I-C, ...","Type,2,diabetes",depression
8,I have gestational diabetes now with my pregna...,"[I, have, gestational, diabetes, now, with, my...","[O, O, I-C, I-C, O, O, O, I-C, O, O, O, I-E, I...","gestational,diabetes","stop,all,the,sugary,drinks"
9,I have gestational diabetes now with my pregna...,"[I, have, gestational, diabetes, now, with, my...","[O, O, I-C, I-C, O, O, O, I-C, O, O, O, I-E, I...",pregnancy,"stop,all,the,sugary,drinks"


In [6]:
########## load FastText vectors #####################
from gensim.models.fasttext import FastText
model = FastText.load(path_we)

In [6]:
# Take random causes + effects to cluster manually
#df_new.sample(n=1000, random_state=0).effects.values.tolist()

In [269]:
# remove stopwords
# lowercase
#manual_clusters = pd.read_excel("/home/adrian/workspace/causality/Causal-associations-diabetes-twitter/data/Causes_effects_clusters.xlsx")
manual_clusters = pd.read_excel("/Users/adrianahne/workspace/PhD/causality/Causal-associations-diabetes-twitter/data/Causes_effects_clusters.xlsx")
manual_clusters = manual_clusters[["Parent name", "Cluster name", "Synonyms"]].dropna(subset=["Cluster name"])
manual_clusters.reset_index(drop=True, inplace=True)

manual_clusters.head(20)

,Parent name,Cluster name,Synonyms
0,Diabetes,diabetes,"diabetic, #diabetic, #diabetes, diabetes melli..."
1,Diabetes,reverse diabetes,NaN
2,Diabetes,T1D,"type 1 diabetes, type 1, #type1, #type1diabete..."
3,Diabetes,T2D,"type 2 diabetes, type 2, #type, #type2diabetes..."
4,Diabetes,diagnosis,NaN
5,Diabetes,management,"control diabetes, uncontrol"
6,Insulin,insulin,"insulin hormone, supplies"
7,Insulin,rationing insulin,"shortage insulin, denying insulin, lack insuli..."
8,Insulin,unable to afford insulin,"can't afford insulin, no access to affordable ..."
9,Insulin,affordable insulin,afford insulin


In [120]:
def addCenterVector(row):
    """ calculates mean (center) vector cluster name and its synonyms """

    vectors = [np.array([model[word] for word in row["Cluster name"]]).mean(axis=0)] # initialise with center vector of cluster name
    #print("\nSynonyms:", row["Synonyms"], type( row["Synonyms"]))
    if not isinstance(row["Synonyms"], float) : # if not nan
        for synonym in row["Synonyms"].split(","):
            filtered_synonym = [word for word in synonym.strip().split(" ") if word not in stopword_list]
            #print("\t",filtered_synonym)
            vectors.append(np.array([model[word] for word in filtered_synonym]).mean(axis=0)) # add mean vector of each synonym (which can be "horrible vision")
            #print(len(vectors))
    cluster_center = np.array(vectors).mean(axis=0)
    return cluster_center

#manual_clusters["center"] = manual_clusters.apply(addCenterVector, axis=1)
#print(manual_clusters.shape)
#manual_clusters.head()

In [202]:
cosine_similarity(model["obesity"].reshape(1,-1), model["Obesity"].reshape(1,-1))

array([[0.5111686]], dtype=float32)

In [10]:
def lemma(syn):
    return [token.string.strip() if token.string.startswith("#") else token.lemma_.strip() for token in nlp(syn)]


def lemmatize(causeOrEffect, mode="causeEffect"):
    """ clusters = False : lemmatization + prep for causes and effects
        clusters = True: lemmatization + prep for clusters + synonyms (have different splits)
    """
    try:

        if isinstance(causeOrEffect, float) :
            return np.nan, np.nan
        else:
            if mode == "causeEffect":
                prep = preprocess(causeOrEffect.strip().split(","))
                causeOrEffect_lemma = lemma(prep)
            elif mode == "clusters":
                prep = preprocess(causeOrEffect.strip().split(" "))
                causeOrEffect_lemma = lemma(prep)
            elif mode == "cluster_synonyms":
                prep = [preprocess(syn.strip().split(" ")) for syn in causeOrEffect.strip().split(",")]
                causeOrEffect_lemma = [lemma(syn) for syn in prep]
            return prep, causeOrEffect_lemma

    except:
        print("causeOrEffect:", causeOrEffect)
        #print("prep:", prep)
    
def preprocess(phrase):
    if isinstance(phrase, float):
        return phrase
    prep = [word for word in phrase if word not in stopwords.words('english') if word != ""]
    return prep




In [11]:
df_new.head()

,text,tokenized,io_tags,causes,effects,cluster_cause,cluster_effect
0,@USER There will be no shortages of insulin or...,"[@USER, There, will, be, no, shortages, of, in...","[O, O, O, O, O, I-C, I-C, I-C, O, O, O, O, O, ...","shortages,of,insulin",worried,,
1,someone 's a type II diabetic and they basical...,"[someone, 's, a, type, II, diabetic, and, they...","[O, O, O, I-C, I-C, I-C, O, O, O, I-E, I-E, I-...","type,II,diabetic","can't,eat,food",,
2,"I would use it to fix my truck , keep bills pa...","[I, would, use, it, to, fix, my, truck, ,, kee...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",diabetes,"deformed,my,feet",,
3,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,"get,rid,of,the,insulin",,
4,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,diet,,


In [12]:
df_new["causes_prep"], df_new["causes_lemma"] = zip(*df_new["causes"].map(lemmatize))
df_new["effects_prep"], df_new["effects_lemma"] = zip(*df_new["effects"].map(lemmatize))

df_new.head()

,text,tokenized,io_tags,causes,effects,cluster_cause,cluster_effect,causes_prep,causes_lemma,effects_prep,effects_lemma
0,@USER There will be no shortages of insulin or...,"[@USER, There, will, be, no, shortages, of, in...","[O, O, O, O, O, I-C, I-C, I-C, O, O, O, O, O, ...","shortages,of,insulin",worried,,,"[shortages, insulin]","[shortages, insulin]",[worried],[worried]
1,someone 's a type II diabetic and they basical...,"[someone, 's, a, type, II, diabetic, and, they...","[O, O, O, I-C, I-C, I-C, O, O, O, I-E, I-E, I-...","type,II,diabetic","can't,eat,food",,,"[type, II, diabetic]","[type, II, diabetic]","[can't, eat, food]","[can't, eat, food]"
2,"I would use it to fix my truck , keep bills pa...","[I, would, use, it, to, fix, my, truck, ,, kee...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",diabetes,"deformed,my,feet",,,[diabetes],[diabetes],"[deformed, feet]","[deform, foot]"
3,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,"get,rid,of,the,insulin",,,[stress],[stress],"[get, rid, insulin]","[get, rid, insulin]"
4,@USER I want to get rid of the insulin too bec...,"[@USER, I, want, to, get, rid, of, the, insuli...","[O, O, O, O, I-E, I-E, I-E, I-E, I-E, O, O, O,...",stress,diet,,,[stress],[stress],[diet],[diet]


In [14]:
manual_clusters = pd.read_excel("/Users/adrianahne/workspace/PhD/causality/Causal-associations-diabetes-twitter/data/Causes_effects_clusters.xlsx")
manual_clusters = manual_clusters[["Parent name", "Cluster name", "Synonyms"]].dropna(subset=["Cluster name"])
manual_clusters.reset_index(drop=True, inplace=True)


manual_clusters["cluster_name_prep"], manual_clusters["cluster_name_lemma"] = zip(*manual_clusters["Cluster name"].map(lambda x: lemmatize(x, mode="clusters")))
manual_clusters["synonym_prep"], manual_clusters["synonym_lemma"] = zip(*manual_clusters["Synonyms"].map(lambda x: lemmatize(x, mode="cluster_synonyms")))


manual_clusters.head(10)

,Parent name,Cluster name,Synonyms,cluster_name_prep,cluster_name_lemma,synonym_prep,synonym_lemma
0,Diabetes,diabetes,"diabetic, #diabetic, #diabetes, diabetes melli...",[diabetes],[diabetes],"[[diabetic], [#diabetic], [#diabetes], [diabet...","[[diabetic], [#diabetic], [#diabetes], [diabet..."
1,Diabetes,reverse diabetes,NaN,"[reverse, diabetes]","[reverse, diabete]",NaN,NaN
2,Diabetes,T1D,"type 1 diabetes, type 1, #type1, #type1diabete...",[T1D],[t1d],"[[type, 1, diabetes], [type, 1], [#type1], [#t...","[[type, 1, diabetes], [type, 1], [#type1], [#t..."
3,Diabetes,T2D,"type 2 diabetes, type 2, #type, #type2diabetes...",[T2D],[T2D],"[[type, 2, diabetes], [type, 2], [#type], [#ty...","[[type, 2, diabetes], [type, 2], [#type], [#ty..."
4,Diabetes,diagnosis,NaN,[diagnosis],[diagnosis],NaN,NaN
5,Diabetes,management,"control diabetes, uncontrol",[management],[management],"[[control, diabetes], [uncontrol]]","[[control, diabetes], [uncontrol]]"
6,Insulin,insulin,"insulin hormone, supplies",[insulin],[insulin],"[[insulin, hormone], [supplies]]","[[insulin, hormone], [supply]]"
7,Insulin,rationing insulin,"shortage insulin, denying insulin, lack insuli...","[rationing, insulin]","[ration, insulin]","[[shortage, insulin], [denying, insulin], [lac...","[[shortage, insulin], [deny, insulin], [lack, ..."
8,Insulin,unable to afford insulin,"can't afford insulin, no access to affordable ...","[unable, afford, insulin]","[unable, afford, insulin]","[[can't, afford, insulin], [access, affordable...","[[can't, afford, insulin], [access, affordable..."
9,Insulin,affordable insulin,afford insulin,"[affordable, insulin]","[affordable, insulin]","[[afford, insulin]]","[[afford, insulin]]"


In [15]:
######### WITH LEMMATIZATION => Create clusters #############

import warnings
warnings.filterwarnings("ignore") 


def calc_cosine_sim(causeOrEffect, causeOrEffect_lemma, cluster_name, cluster_name_lemma):
    """ Test several configurations (lowercased, without 'diabetes') to achieve highest cosine similarity """
    #if causeOrEffect[0] == "stressed" and cluster_name[0] == "stress":
    #    print("\n\tA causeOrEffect:", causeOrEffect, " | causeOrEffect_lemma:", causeOrEffect_lemma, "  | cluster_name:", cluster_name, "  | cluster_name_lemma:", cluster_name_lemma)
    
     
    #s1 = time.time()
    #causeOrEffect_lemma = [token.string.strip() if token.string.startswith("#") else token.lemma_.strip() for token in nlp(causeOrEffect)]
    #global t1
    #t1 += time.time() - s1
    #print("\tcauseOrEffect_lemma:", causeOrEffect_lemma)
    
    s2 = time.time()
    causeOrEffect_vector = np.array([model[word] for word in causeOrEffect]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    global t2
    t2 += time.time() - s2
    #causeOrEffect_lowerCase_vector = np.array([model[word.lower()] for word in causeOrEffect]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    s3 = time.time()
    causeOrEffect_lemma_vector = np.array([model[word] for word in causeOrEffect_lemma]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    global t3
    t3 += time.time() - s3
    
    s4 = time.time()
    causeOrEffect_withoutDiabetes_vector = False
    if len(causeOrEffect) > 1:
        if "diabetic" in causeOrEffect:
            causeOrEffect.remove("diabetic")                
        if "diabetes" in causeOrEffect:
            causeOrEffect.remove("diabetes")
        causeOrEffect_withoutDiabetes_vector = np.array([model[word] for word in causeOrEffect]).mean(axis=0).reshape(1, -1)  # vector of identified cause
        #causeOrEffect_withoutDiabetes_lowerCase_vector = np.array([model[word.lower()] for word in causeOrEffect]).mean(axis=0).reshape(1, -1)  # vector of identified cause
        causeOrEffect_withoutDiabetes_lemma_vector = np.array([model[word] for word in causeOrEffect_lemma]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    global t4
    t4 += time.time() - s4

    #s5 = time.time()
    #cluster_name_lemma = [token.string.strip() if token.string.startswith("#") else token.lemma_.strip() for token in nlp(cluster_name)]
    #global t5
    #t5 += time.time() - s5
    #print("\tcluster_name_lemma:", cluster_name_lemma)
    s6 = time.time()
    cluster_name_vector = np.array([model[word] for word in cluster_name]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    global t6
    t6 += time.time() - s6
    #cluster_name_lower_vector = np.array([model[word.lower()] for word in cluster_name]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    s7 = time.time()
    cluster_name_lemma_vector = np.array([model[word] for word in cluster_name_lemma]).mean(axis=0).reshape(1, -1)  # vector of identified cause
    global t7
    t7 += time.time() - s7
    
    ll = []        
    
    s8 = time.time()
    sim = 1 - cosine(causeOrEffect_vector, cluster_name_vector)
    #if causeOrEffect[0] == "stressed" and cluster_name[0] == "stress":
    #    print("\t\tB sim({}, {}) = ".format(causeOrEffect, cluster_name), sim)
    global t8
    t8 += time.time() - s8
    ll.append( (sim, causeOrEffect, 0) )
    
    #sim = cosine_similarity(causeOrEffect_lowerCase_vector, cluster_name_lower_vector)[0][0] # lowercase
    #ll.append( (sim, causeOrEffect) )
    
    sim = 1 - cosine(causeOrEffect_lemma_vector, cluster_name_lemma_vector)# lemmatization
    ll.append( (sim, causeOrEffect_lemma, 1) )
    #if causeOrEffect[0] == "stressed" and cluster_name[0] == "stress":
    #    print("\t\tC sim({}, {}) = ".format(causeOrEffect_lemma, cluster_name_lemma), sim)

    
    if not isinstance(causeOrEffect_withoutDiabetes_vector, bool):
        sim = 1 - cosine(causeOrEffect_withoutDiabetes_vector, cluster_name_vector)
        ll.append( (sim, causeOrEffect, 2) )      
    #    if causeOrEffect[0] == "stressed" and cluster_name[0] == "stress":
    #        print("\t\tD wo sim({}, {}) = ".format(causeOrEffect, cluster_name), sim)

        #sim = cosine_similarity(causeOrEffect_withoutDiabetes_lowerCase_vector, cluster_name_lower_vector)[0][0]
        #ll.append( (sim, causeOrEffect) )     
        
        sim = 1 - cosine(causeOrEffect_withoutDiabetes_lemma_vector, cluster_name_lemma_vector)
        ll.append( (sim, causeOrEffect_lemma, 3) )            
    #    if causeOrEffect[0] == "stressed" and cluster_name[0] == "stress":
    #        print("\t\tE wo sim({}, {}) = ".format(causeOrEffect_lemma, cluster_name_lemma), sim)

    
    s9 = time.time()

    bestSim, bestCauseOrEffect, bestType = max(ll, key=lambda item:item[0])
    #if causeOrEffect[0] == "stressed" and cluster_name[0] == "stress":
    #    print("\t\tF BEST: ", bestCauseOrEffect, bestSim)
    global t9
    t9 += time.time() - s9
        
        
    if bestType == 0:
        global cc
        cc += 1
    elif bestType == 1:
        global cc_lemma
        cc_lemma += 1
    elif bestType == 2:
        global cc_wo_diab 
        cc_wo_diab += 1
    elif bestType == 3: 
        global cc_wo_diab_lemma
        cc_wo_diab_lemma += 1
        
    return bestSim, causeOrEffect#bestCauseOrEffect


def find_closest_cluster(cause_or_effect, cause_or_effect_prep, cause_or_effect_lemmatized, clusters, t1=None):
#causes = cause_or_effect.strip().split(";")
#causes_lemma = cause_or_effect_lemmatized.strip().split(";")
#for cause, cause_lemma in zip(causes, causes_lemma):
    similarities = []
#    filtered_cause = preprocess(cause.strip().split(","))
    if cause_or_effect_prep: # if non-empty
        for i, row in clusters.iterrows(): # loop over all clusters
            #try:
                if not (isinstance(row["Synonyms"], float) and np.isnan(row["Synonyms"]) ): # if there are synonyms
#                    for synonym in row["Synonyms"].split(","):
                    for synonym, synonym_lemma in zip(row["synonym_prep"], row["synonym_lemma"]):
                        #filtered_synonym = preprocess(synonym.strip().split(" "))
                        sim, causeOrEffect = calc_cosine_sim(cause_or_effect_prep, cause_or_effect_lemmatized, synonym, synonym_lemma)
                        similarities.append((causeOrEffect, synonym, row['Cluster name'], sim))
                      
    
                #filtered_cluster_name = preprocess(row["Cluster name"].strip().split(" "))
                sim, causeOrEffect = calc_cosine_sim(cause_or_effect_prep, cause_or_effect_lemmatized, row["cluster_name_prep"], row["cluster_name_lemma"])
                similarities.append((causeOrEffect, row["cluster_name_prep"], row['Cluster name'], sim))
                #if causeOrEffect[0] == "stressed" and row["cluster_name_prep"][0] == "stress":
                #    print("\t\t Y sim:", sim, "causeOrEffect:", causeOrEffect)
            #except Exception:
            #    print("\nERROR:")
            #    print("filtered_cause:", filtered_cause, "  | filtered_synonym: ", filtered_synonym)
            #    print("cause_or_effect:", cause_or_effect, "  | cause:", cause)


        bestCauseOrEffect, bestClusterSynonym, bestClusterName, bestSim = sorted(similarities, key=lambda tup: tup[3], reverse=True)[0] # take only highest sim
        #if causeOrEffect[0] == "stressed":

        #    for cause, synonym, cluster_name, simil in sorted(similarities, key=lambda tup: tup[3], reverse=True)[0:3]:
        #        print("\tsim({}, {}) = ".format(cause, synonym), simil, "of cluster name:", cluster_name)
        #    print("G RETURN: bestCauseOrEffect:", bestCauseOrEffect, " | bestClusterSynonym:", bestClusterSynonym, " | bestClusterName:", bestClusterName, " | bestSim:", bestSim)
        return bestCauseOrEffect, bestClusterSynonym, bestClusterName, bestSim
    else:
        return "", "", "", -1.0


import time


cc = 0
cc_wo_diab = 0
cc_lemma= 0
cc_wo_diab_lemma = 0
t1 = 0
t2 = 0
t3 = 0
t4 = 0
t5 = 0
t6 = 0
t7 = 0
t8 = 0
t9 = 0


start = time.time()
df_new["cluster_cause"] = ""
df_new["cluster_effect"] = ""
for i, row in df_new.iterrows():
    if i % 1000 == 0:
        print("\n\n ############# i:", i, "(", time.time()-start, "s)", " ##########\n\n")
    cause, cluster_synonym_cause, cluster_name_cause, sim_cause = find_closest_cluster(row["causes"], row["causes_prep"], row["causes_lemma"], manual_clusters)
    #print("\tcause:", cause, " | cluster_name_cause:", cluster_name_cause, " | sim_cause:", sim_cause)
    if sim_cause > 0.0:
        if sim_cause > 0.55: # associate found cluster to the cause; second condition if sim_cause == -1 returned
            df_new.loc[i, "cluster_cause"] = cluster_name_cause
        else: # create new cluster
            new_cluster_prep, new_cluster_lemma = lemmatize(" ".join(cause), mode="clusters")
            manual_clusters.loc[len(manual_clusters.index)] = ["", " ".join(cause), np.nan, new_cluster_prep, new_cluster_lemma, np.nan, np.nan] # Parent name, Cluster name, Synonym
            print("New clusters created (cause):", cause, "  | sim:", sim_cause, " | cluster_name:", cluster_name_cause)
            print("\t\tcause:", cause, " | cluster_name_cause:", cluster_name_cause, " | sim_cause:", sim_cause)

    effect, cluster_synonym_effect, cluster_name_effect, sim_effect = find_closest_cluster(row["effects"], row["effects_prep"], row["effects_lemma"], manual_clusters)    
    #print("\teffect:", effect, " | cluster_name_effect:", cluster_name_effect, " | sim_effect:", sim_effect)
    if sim_effect > 0.0:
        if sim_effect > 0.55: # associate found cluster to the cause; second condition if sim_cause == -1 returned
            df_new.loc[i, "cluster_effect"] = cluster_name_effect
        else: # create new cluster
            new_cluster_prep, new_cluster_lemma = lemmatize(" ".join(effect), mode="clusters")
            manual_clusters.loc[len(manual_clusters.index)] = ["", " ".join(effect), np.nan, new_cluster_prep, new_cluster_lemma, np.nan, np.nan] # Parent name, Cluster name, Synonym
            print("New clusters created (effect):", effect, "  | sim:", sim_effect, " | cluster_name:", cluster_name_effect)    
            print("\t\teffect:", effect, " | cluster_name_effect:", cluster_name_effect, " | sim_effect:", sim_effect)

#  new node for < 0.55
end = time.time()
print("Time:", end-start)
print("cc:", cc, "cc_wo_diab:", cc_wo_diab, "cc_lemma:", cc_lemma, "cc_wo_diab_lemma:", cc_wo_diab_lemma)
print("t1:", t1)
print("t2:", t2)
print("t3:", t3)
print("t4:", t4)
print("t5:", t5)
print("t6:", t6)
print("t7:", t7)
print("t8:", t8)
print("t9:", t9)
# 1000 samples : 6400s
# 300 samples: 865s and 802s in calc cosine function




 ############# i: 0 ( 0.012199878692626953 s)  ##########


New clusters created (effect): ['collect', 'urine', 'sample']   | sim: 0.522433876991272  | cluster_name: insulin pump
		effect: ['collect', 'urine', 'sample']  | cluster_name_effect: insulin pump  | sim_effect: 0.522433876991272
New clusters created (effect): ['stumbling', 'blocks']   | sim: 0.42115408182144165  | cluster_name: insulin spike
		effect: ['stumbling', 'blocks']  | cluster_name_effect: insulin spike  | sim_effect: 0.42115408182144165
New clusters created (effect): ['seizures']   | sim: 0.43465304374694824  | cluster_name: fear
		effect: ['seizures']  | cluster_name_effect: fear  | sim_effect: 0.43465304374694824
New clusters created (effect): ['survive', 'digestion']   | sim: 0.5440065264701843  | cluster_name: fatigue
		effect: ['survive', 'digestion']  | cluster_name_effect: fatigue  | sim_effect: 0.5440065264701843
New clusters created (cause): ['McDonald']   | sim: 0.3946429491043091  | cluster_name: nutrit

New clusters created (effect): ['ass', 'alive']   | sim: 0.5429733991622925  | cluster_name: home
		effect: ['ass', 'alive']  | cluster_name_effect: home  | sim_effect: 0.5429733991622925
New clusters created (cause): ['plummets', '#t1d', '#getthejuiceboxes']   | sim: 0.5226213335990906  | cluster_name: T1D
		cause: ['plummets', '#t1d', '#getthejuiceboxes']  | cluster_name_cause: T1D  | sim_cause: 0.5226213335990906
New clusters created (effect): ['bank', 'account']   | sim: 0.37856730818748474  | cluster_name: insulin prices
		effect: ['bank', 'account']  | cluster_name_effect: insulin prices  | sim_effect: 0.37856730818748474
New clusters created (effect): ['ambulance']   | sim: 0.48328888416290283  | cluster_name: coma
		effect: ['ambulance']  | cluster_name_effect: coma  | sim_effect: 0.48328888416290283
New clusters created (effect): ['smelling', 'fruity']   | sim: 0.5414862036705017  | cluster_name: nutrition
		effect: ['smelling', 'fruity']  | cluster_name_effect: nutrition  | s

New clusters created (effect): ['chill']   | sim: 0.49408355355262756  | cluster_name: overweight
		effect: ['chill']  | cluster_name_effect: overweight  | sim_effect: 0.49408355355262756
New clusters created (effect): ['envious']   | sim: 0.5011279582977295  | cluster_name: fear
		effect: ['envious']  | cluster_name_effect: fear  | sim_effect: 0.5011279582977295
New clusters created (effect): ['Violence', 'begets', 'violence']   | sim: 0.4402768611907959  | cluster_name: theological
		effect: ['Violence', 'begets', 'violence']  | cluster_name_effect: theological  | sim_effect: 0.4402768611907959
New clusters created (effect): ['stop', 'talking']   | sim: 0.5017489790916443  | cluster_name: insulin pump
		effect: ['stop', 'talking']  | cluster_name_effect: insulin pump  | sim_effect: 0.5017489790916443
New clusters created (effect): ['salad']   | sim: 0.5483062267303467  | cluster_name: sugar
		effect: ['salad']  | cluster_name_effect: sugar  | sim_effect: 0.5483062267303467
New cluste

New clusters created (effect): ['rely', 'foodbanks']   | sim: 0.44078612327575684  | cluster_name: diet
		effect: ['rely', 'foodbanks']  | cluster_name_effect: diet  | sim_effect: 0.44078612327575684
New clusters created (cause): ['dietary', 'needs']   | sim: 0.518555760383606  | cluster_name: taking necessary precautions
		cause: ['dietary', 'needs']  | cluster_name_cause: taking necessary precautions  | sim_cause: 0.518555760383606
New clusters created (effect): ['drinking']   | sim: 0.5325705409049988  | cluster_name: sugar
		effect: ['drinking']  | cluster_name_effect: sugar  | sim_effect: 0.5325705409049988
New clusters created (effect): ['fending', 'bankruptcy']   | sim: 0.5181324481964111  | cluster_name: rationing insulin
		effect: ['fending', 'bankruptcy']  | cluster_name_effect: rationing insulin  | sim_effect: 0.5181324481964111
New clusters created (effect): ['shortness', 'breath']   | sim: 0.5144016742706299  | cluster_name: smelling fruity
		effect: ['shortness', 'breath'

New clusters created (effect): ['hurts']   | sim: 0.46915125846862793  | cluster_name: sadness
		effect: ['hurts']  | cluster_name_effect: sadness  | sim_effect: 0.46915125846862793
New clusters created (effect): ['bulging', 'disc']   | sim: 0.45961132645606995  | cluster_name: marching band head
		effect: ['bulging', 'disc']  | cluster_name_effect: marching band head  | sim_effect: 0.45961132645606995
New clusters created (cause): ['weak']   | sim: 0.5004667639732361  | cluster_name: feel bad
		cause: ['weak']  | cluster_name_cause: feel bad  | sim_cause: 0.5004667639732361
New clusters created (effect): ['uneasy']   | sim: 0.3873961269855499  | cluster_name: fear
		effect: ['uneasy']  | cluster_name_effect: fear  | sim_effect: 0.3873961269855499
New clusters created (effect): ['talcum', 'powder']   | sim: 0.44916918873786926  | cluster_name: tomé un vaso de coca cola
		effect: ['talcum', 'powder']  | cluster_name_effect: tomé un vaso de coca cola  | sim_effect: 0.44916918873786926
Ne

...

New clusters created (effect): ['Fantasy', 'Football', 'Manager']   | sim: 0.5046530365943909  | cluster_name: Video Games
		effect: ['Fantasy', 'Football', 'Manager']  | cluster_name_effect: Video Games  | sim_effect: 0.5046530365943909
New clusters created (effect): ['dodgy', ':face_with_thermometer:']   | sim: 0.45153215527534485  | cluster_name: breast removed
		effect: ['dodgy', ':face_with_thermometer:']  | cluster_name_effect: breast removed  | sim_effect: 0.45153215527534485
New clusters created (effect): ['refresh']   | sim: 0.38645368814468384  | cluster_name: hourglass tick
		effect: ['refresh']  | cluster_name_effect: hourglass tick  | sim_effect: 0.38645368814468384
New clusters created (effect): ['destructive', 'cycle']   | sim: 0.5363823175430298  | cluster_name: irregular cycles
		effect: ['destructive', 'cycle']  | cluster_name_effect: irregular cycles  | sim_effect: 0.5363823175430298
New clusters created (cause): ['sour', 'angst']   | sim: 0.5483502745628357  | clust

New clusters created (effect): ['moderate']   | sim: 0.501092255115509  | cluster_name: moderation
		effect: ['moderate']  | cluster_name_effect: moderation  | sim_effect: 0.501092255115509
New clusters created (effect): ['overwhelm', 'lymph', 'glands']   | sim: 0.5088109374046326  | cluster_name: lung infection
		effect: ['overwhelm', 'lymph', 'glands']  | cluster_name_effect: lung infection  | sim_effect: 0.5088109374046326
New clusters created (cause): ['bacteria', 'threatening']   | sim: 0.5412824153900146  | cluster_name: gut
		cause: ['bacteria', 'threatening']  | cluster_name_cause: gut  | sim_cause: 0.5412824153900146
New clusters created (cause): ['silicone', 'poisoning']   | sim: 0.4830162823200226  | cluster_name: ear wax
		cause: ['silicone', 'poisoning']  | cluster_name_cause: ear wax  | sim_cause: 0.4830162823200226
New clusters created (cause): ['OJ']   | sim: 0.42624327540397644  | cluster_name: vodka
		cause: ['OJ']  | cluster_name_cause: vodka  | sim_cause: 0.42624327

New clusters created (effect): [':fire:', ':fire:', ':fire:']   | sim: 0.22834815084934235  | cluster_name: :grinning_cat_face: :pencil:
		effect: [':fire:', ':fire:', ':fire:']  | cluster_name_effect: :grinning_cat_face: :pencil:  | sim_effect: 0.22834815084934235
New clusters created (effect): ['robbed']   | sim: 0.4715302884578705  | cluster_name: vomited infront
		effect: ['robbed']  | cluster_name_effect: vomited infront  | sim_effect: 0.4715302884578705
New clusters created (effect): ['penis', 'envy']   | sim: 0.49729418754577637  | cluster_name: crushed skull
		effect: ['penis', 'envy']  | cluster_name_effect: crushed skull  | sim_effect: 0.49729418754577637


 ############# i: 133000 ( 243635.74712491035 s)  ##########


New clusters created (effect): ['Hahn', 'Super', 'Dry']   | sim: 0.5125468373298645  | cluster_name: Russell Stover
		effect: ['Hahn', 'Super', 'Dry']  | cluster_name_effect: Russell Stover  | sim_effect: 0.5125468373298645
New clusters created (cause): ['jet',

New clusters created (cause): ['arriving', 'warm']   | sim: 0.5430986881256104  | cluster_name: hot steaming
		cause: ['arriving', 'warm']  | cluster_name_cause: hot steaming  | sim_cause: 0.5430986881256104
New clusters created (effect): ['bail']   | sim: 0.38371220231056213  | cluster_name: lashed
		effect: ['bail']  | cluster_name_effect: lashed  | sim_effect: 0.38371220231056213
New clusters created (cause): ['Greek', 'Tina']   | sim: 0.47203412652015686  | cluster_name: Arnold Palmer
		cause: ['Greek', 'Tina']  | cluster_name_cause: Arnold Palmer  | sim_cause: 0.47203412652015686
New clusters created (cause): ['Disabled', 'Vet']   | sim: 0.5496018528938293  | cluster_name: disabled spoonie
		cause: ['Disabled', 'Vet']  | cluster_name_cause: disabled spoonie  | sim_cause: 0.5496018528938293
New clusters created (effect): ['sassy']   | sim: 0.5086559653282166  | cluster_name: grumpy
		effect: ['sassy']  | cluster_name_effect: grumpy  | sim_effect: 0.5086559653282166
New clusters cre



 ############# i: 145000 ( 278416.40447092056 s)  ##########


New clusters created (cause): ['Netflix', 'series']   | sim: 0.5395132899284363  | cluster_name: episode
		cause: ['Netflix', 'series']  | cluster_name_cause: episode  | sim_cause: 0.5395132899284363
New clusters created (effect): [':cooking:', ':bacon:', '#createbeauty', '#createhappiness']   | sim: 0.393110454082489  | cluster_name: :regional_indicator_symbol_letter_s: :regional_indicator_symbol_letter_u: :regional_indicator_symbol_letter_s:
		effect: [':cooking:', ':bacon:', '#createbeauty', '#createhappiness']  | cluster_name_effect: :regional_indicator_symbol_letter_s: :regional_indicator_symbol_letter_u: :regional_indicator_symbol_letter_s:  | sim_effect: 0.393110454082489
New clusters created (cause): ['paycheck']   | sim: 0.35899606347084045  | cluster_name: unable to afford insulin
		cause: ['paycheck']  | cluster_name_cause: unable to afford insulin  | sim_cause: 0.35899606347084045
New clusters created (effect)

In [16]:
manual_clusters.shape

(1915, 7)

In [17]:
manual_clusters.to_csv("/Users/adrianahne/workspace/PhD/causality/Causal-associations-diabetes-twitter/data/all_cause_effect_clusters.csv", sep=";")

In [426]:
#manual_clusters.loc[len(manual_clusters.index)+1] = ["", " ".join(effect), np.nan] # Parent name, Cluster name, Synonym
manual_clusters.head(200)

,Parent name,Cluster name,Synonyms,cluster_name_prep,cluster_name_lemma,synonym_prep,synonym_lemma
0,Diabetes,diabetes,"diabetic, #diabetic, #diabetes, diabetes melli...",[diabetes],[diabetes],"[[diabetic], [#diabetic], [#diabetes], [diabet...","[[diabetic], [#diabetic], [#diabetes], [diabet..."
1,Diabetes,reverse diabetes,NaN,"[reverse, diabetes]","[reverse, diabete]",NaN,NaN
2,Diabetes,T1D,"type 1 diabetes, type 1, #type1, #type1diabete...",[T1D],[t1d],"[[type, 1, diabetes], [type, 1], [#type1], [#t...","[[type, 1, diabetes], [type, 1], [#type1], [#t..."
3,Diabetes,T2D,"type 2 diabetes, type 2, #type, #type2diabetes...",[T2D],[T2D],"[[type, 2, diabetes], [type, 2], [#type], [#ty...","[[type, 2, diabetes], [type, 2], [#type], [#ty..."
4,Diabetes,diagnosis,NaN,[diagnosis],[diagnosis],NaN,NaN
...,...,...,...,...,...,...,...
112,,kicking doors,NaN,"[kicking, doors]","[kick, door]",NaN,NaN
113,,ICU,NaN,[ICU],[ICU],NaN,NaN
114,,dark patch face,NaN,"[dark, patch, face]","[dark, patch, face]",NaN,NaN
115,,DIED,NaN,[DIED],[DIED],NaN,NaN


### associate each cause / effect to its cluster

In [7]:
all_clusters = pd.read_csv("/Users/adrianahne/workspace/PhD/causality/Causal-associations-diabetes-twitter/data/Causes_effects_clusters - automatically derived.csv")
#manual_clusters = manual_clusters[["Parent name", "Cluster name", "Synonyms"]].dropna(subset=["Cluster name"])
#manual_clusters.reset_index(drop=True, inplace=True)
print(all_clusters.shape)
all_clusters.head()


(1751, 8)


,Unnamed: 0,Parent name,Cluster name,Synonyms,cluster_name_prep,cluster_name_lemma,synonym_prep,synonym_lemma
0,0.0,Diabetes,diabetes,"diabetic, #diabetic, #diabetes, diabetes melli...",['diabetes'],['diabetes'],"[['diabetic'], ['#diabetic'], ['#diabetes'], [...","[['diabetic'], ['#diabetic'], ['#diabetes'], [..."
1,1.0,Diabetes,reverse diabetes,"reversed, cured overnight\n","['reverse', 'diabetes']","['reverse', 'diabete']",NaN,NaN
2,2.0,Diabetes,T1D,"type 1 diabetes, type 1, #type1, #type1diabete...",['T1D'],['t1d'],"[['type', '1', 'diabetes'], ['type', '1'], ['#...","[['type', '1', 'diabetes'], ['type', '1'], ['#..."
3,3.0,Diabetes,T2D,"type 2 diabetes, type 2, #type, #type2diabetes...",['T2D'],['T2D'],"[['type', '2', 'diabetes'], ['type', '2'], ['#...","[['type', '2', 'diabetes'], ['type', '2'], ['#..."
4,4.0,Diabetes,diagnosis,NaN,['diagnosis'],['diagnosis'],NaN,NaN


In [18]:
all_clusters["cluster_name_prep"], all_clusters["cluster_name_lemma"] = zip(*all_clusters["Cluster name"].map(lambda x: lemmatize(x, mode="clusters")))
all_clusters["synonym_prep"], all_clusters["synonym_lemma"] = zip(*all_clusters["Synonyms"].map(lambda x: lemmatize(x, mode="cluster_synonyms")))
all_clusters.head()


,Unnamed: 0,Parent name,Cluster name,Synonyms,cluster_name_prep,cluster_name_lemma,synonym_prep,synonym_lemma
0,0.0,Diabetes,diabetes,"diabetic, #diabetic, #diabetes, diabetes melli...",[diabetes],[diabetes],"[[diabetic], [#diabetic], [#diabetes], [diabet...","[[diabetic], [#diabetic], [#diabetes], [diabet..."
1,1.0,Diabetes,reverse diabetes,"reversed, cured overnight\n","[reverse, diabetes]","[reverse, diabete]","[[reversed], [cured, overnight]]","[[reversed], [cure, overnight]]"
2,2.0,Diabetes,T1D,"type 1 diabetes, type 1, #type1, #type1diabete...",[T1D],[t1d],"[[type, 1, diabetes], [type, 1], [#type1], [#t...","[[type, 1, diabetes], [type, 1], [#type1], [#t..."
3,3.0,Diabetes,T2D,"type 2 diabetes, type 2, #type, #type2diabetes...",[T2D],[T2D],"[[type, 2, diabetes], [type, 2], [#type], [#ty...","[[type, 2, diabetes], [type, 2], [#type], [#ty..."
4,4.0,Diabetes,diagnosis,NaN,[diagnosis],[diagnosis],NaN,NaN


In [35]:
import warnings
warnings.filterwarnings("ignore") 


def calc_cosine_sim(causeOrEffect, causeOrEffect_lemma, cluster_name, cluster_name_lemma):
    """ Test several configurations (lowercased, without 'diabetes') to achieve highest cosine similarity """

    causeOrEffect_vector = np.array([model[word] for word in causeOrEffect]).mean(axis=0)#.reshape(1, -1)  # vector of identified cause
    causeOrEffect_lemma_vector = np.array([model[word] for word in causeOrEffect_lemma]).mean(axis=0)#.reshape(1, -1)  # vector of identified cause
    
    causeOrEffect_withoutDiabetes_vector = False
    if len(causeOrEffect) > 1:
        if "diabetic" in causeOrEffect:
            causeOrEffect.remove("diabetic")                
        if "diabetes" in causeOrEffect:
            causeOrEffect.remove("diabetes")
        causeOrEffect_withoutDiabetes_vector = np.array([model[word] for word in causeOrEffect]).mean(axis=0)#.reshape(1, -1)  # vector of identified cause
        causeOrEffect_withoutDiabetes_lemma_vector = np.array([model[word] for word in causeOrEffect_lemma]).mean(axis=0)#.reshape(1, -1)  # vector of identified cause


    cluster_name_vector = np.array([model[word] for word in cluster_name]).mean(axis=0)#.reshape(1, -1)  # vector of identified cause
    cluster_name_lemma_vector = np.array([model[word] for word in cluster_name_lemma]).mean(axis=0)#.reshape(1, -1)  # vector of identified cause
    
    ll = []        
    sim = np.dot(causeOrEffect_vector, cluster_name_vector) / ( np.linalg.norm(causeOrEffect_vector) * np.linalg.norm(cluster_name_vector) ) #1 - cosine(causeOrEffect_vector, cluster_name_vector)
    ll.append( (sim, causeOrEffect, 0) )

    sim = np.dot(causeOrEffect_lemma_vector, cluster_name_lemma_vector) / ( np.linalg.norm(causeOrEffect_lemma_vector) * np.linalg.norm(cluster_name_lemma_vector) ) #1 - cosine(causeOrEffect_lemma_vector, cluster_name_lemma_vector)# lemmatization
    ll.append( (sim, causeOrEffect_lemma, 1) )
    
    if not isinstance(causeOrEffect_withoutDiabetes_vector, bool):
        sim = np.dot(causeOrEffect_withoutDiabetes_vector, cluster_name_vector) / ( np.linalg.norm(causeOrEffect_withoutDiabetes_vector) * np.linalg.norm(cluster_name_vector) ) #1 - cosine(causeOrEffect_withoutDiabetes_vector, cluster_name_vector)
        ll.append( (sim, causeOrEffect, 2) )      
    
        sim = np.dot(causeOrEffect_withoutDiabetes_lemma_vector,cluster_name_lemma_vector) / ( np.linalg.norm(causeOrEffect_withoutDiabetes_lemma_vector) * np.linalg.norm(cluster_name_lemma_vector) ) #1 - cosine(causeOrEffect_withoutDiabetes_lemma_vector, cluster_name_lemma_vector)
        ll.append( (sim, causeOrEffect_lemma, 3) )            

    bestSim, bestCauseOrEffect, bestType = max(ll, key=lambda item:item[0])


    return bestSim, causeOrEffect#bestCauseOrEffect


def find_closest_cluster(cause_or_effect, cause_or_effect_prep, cause_or_effect_lemmatized, clusters, t1=None):

    similarities = []
    if cause_or_effect_prep: # if non-empty
        for i, row in clusters.iterrows(): # loop over all clusters
                if not (isinstance(row["Synonyms"], float) and np.isnan(row["Synonyms"]) ): # if there are synonyms
                    if (isinstance(row["synonym_prep"], float) or isinstance(row["synonym_lemma"], float) ):
                        print(row["Synonyms"])
                        print(row["synonym_prep"])
                        print(row["synonym_lemma"])
                    for synonym, synonym_lemma in zip(row["synonym_prep"], row["synonym_lemma"]):
                        sim, causeOrEffect = calc_cosine_sim(cause_or_effect_prep, cause_or_effect_lemmatized, synonym, synonym_lemma)
                        similarities.append((causeOrEffect, synonym, row['Cluster name'], sim))
                      
                sim, causeOrEffect = calc_cosine_sim(cause_or_effect_prep, cause_or_effect_lemmatized, row["cluster_name_prep"], row["cluster_name_lemma"])
                similarities.append((causeOrEffect, row["cluster_name_prep"], row['Cluster name'], sim))

        bestCauseOrEffect, bestClusterSynonym, bestClusterName, bestSim = sorted(similarities, key=lambda tup: tup[3], reverse=True)[0] # take only highest sim
        return bestCauseOrEffect, bestClusterSynonym, bestClusterName, bestSim
    else:
        return "", "", "", -1.0


import time


start = time.time()
df_new["cluster_cause"] = ""
df_new["cluster_effect"] = ""
for j in np.arange(2000, 50000, 2000):
    for i, row in df_new[j:j+2000].iterrows():
        if i % 1000 == 0:
            print("\n\n ############# j:", j,"| i:", i, "(", time.time()-start, "s)", " ##########\n\n")
        cause, cluster_synonym_cause, cluster_name_cause, sim_cause = find_closest_cluster(row["causes"], row["causes_prep"], row["causes_lemma"], all_clusters)
        #print("\n\tcause:", cause, " | cluster_name_cause:", cluster_name_cause, " | sim_cause:", sim_cause)
        #if sim_cause > 0.0:
        if sim_cause > 0.55: # associate found cluster to the cause; second condition if sim_cause == -1 returned
            df_new.loc[i, "cluster_cause"] = cluster_name_cause

        else: # create new cluster
            print("No close cluster?? cause:", cause, "  | sim:", sim_cause, " | cluster_name:", cluster_name_cause)

        effect, cluster_synonym_effect, cluster_name_effect, sim_effect = find_closest_cluster(row["effects"], row["effects_prep"], row["effects_lemma"], all_clusters)    
        #print("\teffect:", effect, " | cluster_name_effect:", cluster_name_effect, " | sim_effect:", sim_effect)
        #if sim_effect > 0.0:
        if sim_effect > 0.55: # associate found cluster to the cause; second condition if sim_cause == -1 returned
            df_new.loc[i, "cluster_effect"] = cluster_name_effect
        else: # create new cluster
            print("No close cluster ?? effect:", effect, "  | sim:", sim_effect, " | cluster_name:", cluster_name_effect)    
    
    df_new[j:j+2000].to_csv("network_predictions/tweets_clusters_{}_{}.csv".format(j, j+2000), sep=";")
    
#  new node for < 0.55
end = time.time()
print("Time:", end-start)




 ############# j: 2000 | i: 2000 ( 0.009665966033935547 s)  ##########




 ############# j: 2000 | i: 3000 ( 2828.6509211063385 s)  ##########




 ############# j: 4000 | i: 4000 ( 35143.51403713226 s)  ##########




 ############# j: 4000 | i: 5000 ( 48263.59558105469 s)  ##########


No close cluster ?? effect:    | sim: -1.0  | cluster_name: 


 ############# j: 6000 | i: 6000 ( 50965.338816165924 s)  ##########


No close cluster ?? effect:    | sim: -1.0  | cluster_name: 


 ############# j: 6000 | i: 7000 ( 54146.92596197128 s)  ##########


No close cluster?? cause:    | sim: -1.0  | cluster_name: 
No close cluster ?? effect:    | sim: -1.0  | cluster_name: 


 ############# j: 8000 | i: 8000 ( 57706.51382398605 s)  ##########


No close cluster ?? effect:    | sim: -1.0  | cluster_name: 
No close cluster ?? effect: ['costs', '1400', 'dollars']   | sim: 0.5255981  | cluster_name: finance
No close cluster?? cause:    | sim: -1.0  | cluster_name: 


 ############# j: 8000 | i

In [25]:

    
df_new[1995:2005].head(10)

,text,tokenized,io_tags,causes,effects,cluster_cause,cluster_effect,causes_prep,causes_lemma,effects_prep,effects_lemma
1995,I have diabetes and I 'm voting for Biden so T...,"[I, have, diabetes, and, I, 'm, voting, for, B...","[O, O, I-C, O, O, O, O, O, O, O, O, O, O, O, O...",diabetes,insulin,diabetes,insulin,[diabetes],[diabetes],[insulin],[insulin]
1996,I have diabetes and I 'm voting for Biden so T...,"[I, have, diabetes, and, I, 'm, voting, for, B...","[O, O, I-C, O, O, O, O, O, O, O, O, O, O, O, O...",diabetes,suffering,diabetes,suffer,[diabetes],[diabetes],[suffering],[suffering]
1997,"@USER My money is on """""""" he died of ( pick on...","[@USER, My, money, is, on, """", he, died, of, (...","[O, O, O, O, O, O, O, I-E, O, O, O, O, O, O, O...",diabetes,died,diabetes,death,[diabetes],[diabetes],[died],[die]
1998,"@USER My money is on """""""" he died of ( pick on...","[@USER, My, money, is, on, """", he, died, of, (...","[O, O, O, O, O, O, O, I-E, O, O, O, O, O, O, O...",hypertension,died,hypertension,death,[hypertension],[hypertension],[died],[die]
1999,it 's bad enough corona hitting the black comm...,"[it, 's, bad, enough, corona, hitting, the, bl...","[O, O, O, O, O, O, O, O, O, O, I-E, I-E, I-E, ...",diabetes,"pre,existing,conditions",diabetes,prediabetes,[diabetes],[diabetes],"[pre, existing, conditions]","[pre, exist, condition]"
2000,it 's bad enough corona hitting the black comm...,"[it, 's, bad, enough, corona, hitting, the, bl...","[O, O, O, O, O, O, O, O, O, O, I-E, I-E, I-E, ...",diabetes,obesity,,,[diabetes],[diabetes],[obesity],[obesity]
2001,@USER My wife is at high risk of serious compl...,"[@USER, My, wife, is, at, high, risk, of, seri...","[O, O, O, O, O, O, O, O, O, I-E, O, O, O, I-C,...","type,one,diabetic",complications,,,"[type, one, diabetic]","[type, one, diabetic]",[complications],[complication]
2002,@USER My wife is at high risk of serious compl...,"[@USER, My, wife, is, at, high, risk, of, seri...","[O, O, O, O, O, O, O, O, O, I-E, O, O, O, I-C,...","type,one,diabetic",fear,,,"[type, one, diabetic]","[type, one, diabetic]",[fear],[fear]
2003,Just look at this thread of diabetics who 've ...,"[Just, look, at, this, thread, of, diabetics, ...","[O, O, O, O, O, O, O, O, O, I-E, O, O, O, O, I...","withheld,insulin",died,,,"[withheld, insulin]","[withheld, insulin]",[died],[die]
2004,God is Good my uncle that had vivid 19 in a di...,"[God, is, Good, my, uncle, that, had, vivid, 1...","[O, O, O, O, O, O, O, O, O, O, O, I-C, I-C, O,...","diabetic,coma","brain,dead",,,"[diabetic, coma]","[diabetic, coma]","[brain, dead]","[brain, dead]"


In [26]:
df_new[0:2000].to_csv("network_predictions/tweets_clusters_0_2000.csv", sep=";")

In [439]:
df_new.sample(n=300, random_state=9).head(20)

,text,tokenized,io_tags,causes,effects,causes_prep,causes_lemma,effects_prep,effects_lemma,cluster_cause,cluster_effect
4413,when i was diagnosed w diabetes my friends tol...,"[when, i, was, diagnosed, w, diabetes, my, fri...","[O, O, O, O, O, I-C, O, O, O, O, O, O, O, O, I...",diabetes,"lot,of,pressure",[diabetes],[diabetes],"[lot, pressure]","[lot, pressure]",diabetes,hypotension
92975,Having to manage 3 cases of stomatitis and now...,"[Having, to, manage, 3, cases, of, stomatitis,...","[O, O, O, O, O, O, O, O, O, O, I-C, O, O, O, O...",diabetic,shock,[diabetic],[diabetic],[shock],[shock],diabetes,shock
15197,@USER Explain how when seeing a annual eye tes...,"[@USER, Explain, how, when, seeing, a, annual,...","[O, O, O, O, O, O, O, I-E, I-E, O, O, O, O, O,...","#T1D,#diabetes","eye,test","[#T1D, #diabetes]","[#T1D, #diabetes]","[eye, test]","[eye, test]",diabetes,OGTT
26659,I read a post this morning about a young lady ...,"[I, read, a, post, this, morning, about, a, yo...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","cost,of,insulin",stressed,"[cost, insulin]","[cost, insulin]",[stressed],[stress],insulin prices,stress
98008,Some months ago im taking a pill for my insuli...,"[Some, months, ago, im, taking, a, pill, for, ...","[O, O, O, O, O, O, O, O, O, I-E, O, O, O, O, I...",diabetes,insuline,[diabetes],[diabetes],[insuline],[insuline],diabetes,insulin
128256,"People can complain about depression , anxiety...","[People, can, complain, about, depression, ,, ...","[O, O, O, O, I-E, O, O, O, O, I-E, O, O, O, O,...","type,1,diabetic",sick,"[type, 1]","[type, 1, diabetic]",[sick],[sick],T1D,sick
29321,Off to Wal Mart on my way home from work to pi...,"[Off, to, Wal, Mart, on, my, way, home, from, ...","[O, O, O, O, O, O, O, O, O, O, O, I-C, I-C, I-...","pick,up,diabetic,foot,cream",bread,"[pick, foot, cream]","[pick, diabetic, foot, cream]",[bread],[bread],legs swollen,nutrition
64793,My glucose test is next week maybe I should st...,"[My, glucose, test, is, next, week, maybe, I, ...","[O, I-C, I-C, O, O, O, O, O, O, I-E, I-E, O, O...","glucose,test","start,fasting","[glucose, test]","[glucose, test]","[start, fasting]","[start, fast]",OGTT,fasting
127780,I 'm doing a blood glucose curve on Pais today...,"[I, 'm, doing, a, blood, glucose, curve, on, P...","[O, O, O, O, I-C, I-C, I-C, O, O, O, O, O, I-E...","blood,glucose,curve","checking,his,BG","[blood, glucose, curve]","[blood, glucose, curve]","[checking, BG]","[check, BG]",hypo,glucose
76385,I 've lost several members of my family to typ...,"[I, 've, lost, several, members, of, my, famil...","[O, O, I-E, I-E, I-E, O, O, O, O, I-C, I-C, I-...","severe,obesity","lost,several,members","[severe, obesity]","[severe, obesity]","[lost, several, members]","[lose, several, member]",overweight,neuropathy


In [460]:
!conda install scipy -y

Solving environment: done

# All requested packages already installed.



In [479]:
import numpy as np
import scipy.sparse as sp
from scipy.spatial.distance import squareform, pdist, cosine, cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

# Create an adjacency matrix
np.random.seed(42)
a = np.random.randint(0, 10, (1,10000)).ravel()
b = np.random.randint(0, 10, (1,10000)).ravel()


# Define a function to calculate the cosine similarities a few different ways
def calc_sim(a,b, method=1):
    if method == 1:
        return 1 - cosine(a,b)
    if method == 2:
        
        return np.dot(a,b) / ( np.linalg.norm(a) * np.linalg.norm(b) )
    if method == 3:
        return cosine_similarity(a.reshape(1,-1),b.reshape(1,-1))
    if method == 4:
        return(1 - cosine_similarity(a,b))


# Time them:
print("Method 1")
%timeit calc_sim(a,b, method=1)
print("Method 2")
%timeit calc_sim(a,b, method=2)
print("Method 3")
%timeit calc_sim(a,b, method=3)
print("Method 4")
%timeit calc_sim(a,b, method=4)

Method 1
114 µs ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Method 2
59.6 µs ± 5.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Method 3
417 µs ± 97.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Method 4


NameError: name 'spatial' is not defined

In [33]:
for i in np.arange(0, 50000, 2000):
    print(i)

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000


In [ ]:
np.arange()